In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Nested Fork-Join Network
model = Network('model')

In [3]:
# Create nested fork-join structure
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)
queue3 = Queue(model,'Queue3',SchedStrategy.FCFS)
queue4 = Queue(model,'Queue4',SchedStrategy.FCFS)
fork2 = Fork(model,'Fork2')
join2 = Join(model,'Join2', fork2)

jobclass1 = ClosedClass(model, 'class1', 1, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
queue3.set_service(jobclass1, Exp(2.0))
queue4.set_service(jobclass1, Exp(2.0))

In [5]:
# Nested fork-join routing
P = model.init_routing_matrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
# Nested structure: queue1 goes to fork2
P.set(jobclass1, jobclass1, queue1, fork2, 1.0)
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, join, 1.0)
# Queue2 goes directly to join
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000)]
# Add MVA with special fork-join options
mva_options = SolverMVA.defaultOptions()
mva_options.config.fork_join = 'fjt'
solvers.append(SolverMVA(model, mva_options))

for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/1135324105097683643/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 0.994562s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  0.5038  0.5038  1.9924  1.9924  0.2505  0.2496
1  Queue1   class1  0.2431  0.2431  0.9990  0.4995  0.2496  0.2496
2  Queue2   class1  0.2518  0.2518  0.9906  0.4953  0.2496  0.2496
3    Join   class1  0.3067  0.0000  0.6164  0.6164  0.5051  0.2496
4  Queue3   class1  0.1247  0.1247  0.4897  0.1224  0.2496  0.2496
5  Queue4   class1  0.1226  0.1226  0.4926  0.1232  0.2496  0.2496
6   Join2   class1  0.1258  0.0000  0.2453  0.1227  0.5001  0.2496
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.059348s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  0.4563  0.4563  2.0000  2.0000  0.2282  0.2282
1  Queue1   class1  0.2762  0.2281  1.2109  0.6054  0.1141  0.2281
2